In [ ]:
!pip install -q -U accelerate bitsandbytes git+https://github.com/huggingface/transformers.git
!pip install datasets -q
!pip install peft -q

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
from transformers import AutoProcessor, PaliGemmaConfig, PaliGemmaForConditionalGeneration
from PIL import Image
import requests
import torch

In [3]:
model_id = "google/paligemma-3b-mix-224"

In [4]:
config = PaliGemmaConfig.from_pretrained(model_id)
config.hidden_activation = 'gelu_pytorch_tanh'

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, config=config)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
processor = AutoProcessor.from_pretrained(model_id, config=config)

In [14]:
# Input 1: The prompt
prompt = "What is on the flower?"

# Input 2: The image
image_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg?download=true"
image = Image.open(requests.get(image_url, stream=True).raw)

# Encode the inputs
input_encoded = processor(prompt, image, return_tensors="pt")
input_encoded_length = inputs["input_ids"].shape[-1] 

with torch.inference_mode():
    # Perform the inference
    outputs = model.generate(**inputs, max_new_tokens=20, do_sample=False)
    output = outputs[0][inputs_length:] # filter out the prompt

    # Decode the output
    output_decoded = processor.decode(output, skip_special_tokens=True)
    print(output_decoded)

bee
